# PFC: $\bigtriangleup \leftrightarrow \square$

The free energy functional is:

$$
\begin{align}
\mathcal{F} = \int_V d\mathbf{r}\left[\frac{n^2(\mathbf{r})}{2}-\eta\frac{n^3(\mathbf{r})}{6}+\chi\frac{n^4(\mathbf{r})}{12}-\frac{1}{2}n(\mathbf{r})\int_V d\mathbf{r}'\left[C_2(\mathbf{r}-\mathbf{r}')n(\mathbf{r}')\right]\right]
\end{align}
$$

the dynamics is:

$$
\begin{align}
\frac{\partial n}{\partial t} &= M\nabla^2\frac{\delta \mathcal{F}}{\delta n} \\
&= M\nabla^2\left[n-\frac{n^2}{2}+\frac{n^3}{3}-\mathfrak{F}^{-1}\left[\hat{C}\cdot\hat{n}\right]\right] \\
&\Downarrow f_{nl} = \frac{n^2}{2}-\frac{n^3}{3} +\mathfrak{F}^{-1}\left[\hat{C}\cdot\hat{n}\right] \nonumber \\
\frac{\hat{n}^{t+1}-\hat{n}^{t}}{\Delta t} &= -Mk^2\left[\hat{n}^{t+1}-\hat{f}_{nl}^{t}\right] \\
&\Downarrow \nonumber \\
(1+\Delta tMk^2)\hat{n}^{t+1} &= \hat{n}^{t} + \Delta tMk^2\hat{f}_{nl}^{t}
\end{align}
$$

In [45]:
import cupy as cp
import numpy as np
from pyevtk.hl import gridToVTK
import matplotlib.pyplot as plt

def ksComp(lx,ly,dx,dy):
    """ Compute the k-space elements

    Args:
        lx (int): number of elements along x
        ly (int): number of elements along y
        dx (float): length step along x
        dy (float): length step along y
    """
    kxl = cp.fft.fftfreq(lx,d=dx/(2*cp.pi)) # kx list
    kyl = cp.fft.rfftfreq(ly,d=dy/(2*cp.pi)) # ky list
    [kxm, kym]=cp.meshgrid(kxl, kyl,indexing='ij') # kx, ky matrices
    k2m = kxm**2 + kym**2 # k^2 matrix
    k1m = cp.sqrt(k2m)    # k matrix
    k4m = k2m**2          # k^4 matrix
    return(kxm, kym, k1m, k2m, k4m)

def CkComp(ks, sigma, alpha1, alpha2):
    Ck1 = cp.exp(-sigma**2/8.0)*cp.exp(-(ks-2*cp.pi)**2/alpha1**2)
    Ck2 = cp.exp(-sigma**2/5.657)*cp.exp(-(ks-2*cp.pi/cp.sqrt(2))**2/alpha2**2)
    Ck = cp.max(cp.stack([Ck1, Ck2], axis=2), axis=2)
    return(Ck)

lx = 256        # length on x
ly = 256        # length on y
dx = dy = 0.2   # unit length
n0 = 0.04       # initial density average
fluc = 0.05     # initial densiity deviation
sigma = 0.4     # effective temperature
alpha1 = alpha2 = 1.0 # interface
M = 0.8         # mobility
dt = 0.1        # time step
tsteps = 5000   # simulation total steps
dsteps = 250    # dump data every dsteps
nsteps = 200    # nucleation steps (not used in this code)

n = cp.random.normal(n0, fluc, (lx, ly))    # initialize
(kx, ky, ks, k2, k4) = ksComp(lx,ly,dx,dy)
Ck = CkComp(ks, sigma, alpha1, alpha2)
fname = 'low2d'

for t in range(tsteps):
    nk = cp.fft.rfft2(n)
    fnl = n**2/2.0 - n**3/3.0 + cp.fft.irfft2(Ck*nk)
    fnlk = cp.fft.rfft2(fnl)
    nom = nk + dt*M*k2*fnlk
    denom = 1 + dt*M*k2
    nk = nom/denom
    n = cp.fft.irfft2(nk) + cp.random.normal(0,fluc, (lx, ly))
    if (np.mod(t, dsteps) == 0):
        n_3d = cp.asnumpy(n[cp.newaxis, :, :])
        gridToVTK(fname+'.'+str(t), x=np.arange(ly)*dy, y=np.arange(lx)*dx, z=np.array([0.0]), pointData = {'composition': n_3d})

for a 3D case:

In [55]:
import cupy as cp
import numpy as np
from pyevtk.hl import gridToVTK
import matplotlib.pyplot as plt

def ksComp(lx,ly,lz,dx,dy,dz):
    """ Compute the k-space elements

    Args:
        lx (int): number of elements along x
        ly (int): number of elements along y
        lz (int): number of elements along z
        dx (float): length step along x
        dy (float): length step along y
        dz (float): length step along z
    """
    kxl = cp.fft.fftfreq(lx,d=dx/(2*np.pi)) # kx list
    kyl = cp.fft.fftfreq(ly,d=dy/(2*np.pi)) # ky list
    kzl = cp.fft.rfftfreq(lz,d=dz/(2*np.pi)) # kz list
    [kxm, kym, kzm]=cp.meshgrid(kxl, kyl, kzl, indexing='ij') # kx, ky, kz matrices
    k2m = kxm**2 + kym**2 + kzm**2  # k^2 matrix
    k1m = cp.sqrt(k2m)              # k^1 matrix
    k4m = k2m**2   # k^4 matrix
    return(kxm, kym, kzm, k1m, k2m, k4m)

def CkComp(ks, sigma, alpha1, alpha2):
    Ck1 = cp.exp(-sigma**2/8.0)*cp.exp(-(ks-2*cp.pi)**2/alpha1**2)
    Ck2 = cp.exp(-sigma**2/5.657)*cp.exp(-(ks-2*cp.pi/cp.sqrt(2))**2/alpha2**2)
    Ck = cp.max(cp.stack([Ck1, Ck2], axis=3), axis=3)
    return(Ck)

lx = 128        # length on x
ly = 128        # length on y
lz = 128        # length on z
dx = dy = dz = 0.2   # unit length
n0 = 0.09       # initial density average
fluc = 0.02     # initial densiity deviation
sigma = 0.12    # effective temperature
alpha1 = alpha2 = 1.0 # interface
M = 1.00        # mobility
dt = 0.1        # time step
tsteps = 8000   # simulation total steps
dsteps = 250    # dump data every dsteps
nsteps = 200    # nucleation steps (not used in this code)

n = cp.random.normal(n0, fluc, (lx, ly))    # initialize
(kx, ky, kz, ks, k2, k4) = ksComp(lx,ly,lz,dx,dy,dz)
Ck = CkComp(ks, sigma, alpha1, alpha2)
fname = 'fcc'

for t in range(tsteps):
    nk = cp.fft.rfftn(n)
    fnl = n**2/2.0 - n**3/3.0 + cp.fft.irfftn(Ck*nk)
    fnlk = cp.fft.rfftn(fnl)
    nom = nk + dt*M*k2*fnlk
    denom = 1 + dt*M*k2
    nk = nom/denom
    n = cp.fft.irfftn(nk) + cp.random.normal(0,fluc, (lx, ly, lz))
    if (np.mod(t, dsteps) == 0):
        n_3d = cp.asnumpy(n)
        gridToVTK(fname+'.'+str(t), x=np.arange(lz)*dz, y=np.arange(ly)*dy, z=np.arange(lx)*dx, pointData = {'density': n_3d})

the simulated structures are:

<img src="./image/bcc.gif" width="800" alt="p1">

and 

<img src="./image/fcc.gif" width="800" alt="p1">